# Knapsack Problem
Castillo, Anjelica M. <br>
MCSCC 123-MSCS12S1 - Advanced Data Structures and Algorithms

### Define the Problem
The problem is, crates can’t weight more than 1000 kilograms, and that’s a hard limit. To make things harder, I can choose only from a limited set of things, already packed in boxes: 


In [19]:
import itertools
import numpy as np
import pandas as pd

define_items = [
    {"name": "Potatoes", "weight": 800, "calories": 1501600},
    {"name": "Wheat flour", "weight": 400, "calories": 1444000},
    {"name": "Rice", "weight": 300, "calories": 1122000},
    {"name": "Beans (can)", "weight": 300, "calories": 690000},
    {"name": "Tomatoes (can)", "weight": 300, "calories": 237000},
    {"name": "Strawberry jam", "weight": 50, "calories": 130000},
    {"name": "Peanut butter", "weight": 20, "calories": 117800}
]
MAX_WEIGHT = 1000 #The max_weight allowed

### Bruteforce Approach

In [20]:
def brute_force_knapsack(items, max_weight):
    """Solves the problem using brute force."""
    best_value = 0
    best_combination = None
    for r in range(len(items) + 1):
        for combination in itertools.combinations(items, r):
            total_weight = sum(item[1] for item in combination)
            total_value = sum(item[2] for item in combination)
            if total_weight <= max_weight and total_value > best_value:
                best_value = total_value
                best_combination = combination
    return best_combination, best_value

brute_force_result = brute_force_knapsack(items, max_weight)
print("Brute Force Solution:", brute_force_result)

Brute Force Solution: ((('Wheat flour', 400, 1444000), ('Rice', 300, 1122000), ('Beans (can)', 300, 690000)), 3256000)


### Dynamic Approach

In [21]:

def dynamic_knapsack(items, max_weight):
    """Solves the problem using dynamic programming."""
    n = len(items)
    dp = np.zeros((n + 1, max_weight + 1))
    for i in range(1, n + 1):
        for w in range(max_weight + 1):
            if items[i - 1][1] <= w:
                dp[i][w] = max(dp[i - 1][w], dp[i - 1][w - items[i - 1][1]] + items[i - 1][2])
            else:
                dp[i][w] = dp[i - 1][w]
    return dp[n][max_weight]

dp_result = dynamic_knapsack(items, max_weight)
print("Dynamic Programming Solution:", dp_result)

Dynamic Programming Solution: 3256000.0


### Fractional Approach

In [22]:
def fractional_knapsack(items, max_weight):
    """Solves the problem using a greedy approach."""
    items_sorted = sorted(items, key=lambda x: x[2] / x[1], reverse=True)
    total_value = 0
    selected_items = []
    for item in items_sorted:
        if max_weight >= item[1]:
            selected_items.append((item[0], item[1], item[2]))
            max_weight -= item[1]
            total_value += item[2]
        else:
            fraction = max_weight / item[1]
            selected_items.append((item[0], max_weight, item[2] * fraction))
            total_value += item[2] * fraction
            break
    return selected_items, total_value

fractional_result = fractional_knapsack(items, max_weight)
print("Fractional Knapsack Solution:", fractional_result)

Fractional Knapsack Solution: ([('Peanut butter', 20, 117800), ('Rice', 300, 1122000), ('Wheat flour', 400, 1444000), ('Strawberry jam', 50, 130000), ('Beans (can)', 230, 529000.0)], 3342800.0)


### Reduce Weight Approach

In [23]:
def reduce_weight_knapsack(items, max_weight, reduce_factor=0.5):
    """Solves a problem where we can take partial weights of each item."""
    adjusted_items = [(item[0], int(item[1] * reduce_factor), int(item[2] * reduce_factor)) for item in items]
    return fractional_knapsack(adjusted_items, max_weight)

# Define items as (name, weight, calories)
items = [
    ("Potatoes", 800, 1501600),
    ("Wheat flour", 400, 1444000),
    ("Rice", 300, 1122000),
    ("Beans (can)", 300, 690000),
    ("Tomatoes (can)", 300, 237000),
    ("Strawberry jam", 50, 130000),
    ("Peanut butter", 20, 117800)
]

max_weight = 1000

reduced_weight_result = reduce_weight_knapsack(items, max_weight)
print("Reduced Weight Knapsack Solution:", reduced_weight_result)

Reduced Weight Knapsack Solution: ([('Peanut butter', 10, 58900), ('Rice', 150, 561000), ('Wheat flour', 200, 722000), ('Strawberry jam', 25, 65000), ('Beans (can)', 150, 345000), ('Potatoes', 400, 750800), ('Tomatoes (can)', 65, 51350.0)], 2554050.0)


### Output & Comparison

In [24]:
# Create DataFrame to visualize results
data = {
    "Approach": ["Brute Force", "Dynamic Programming", "Fractional Knapsack", "Reduced Weight Knapsack"],
    "Total Calories": [brute_force_result[1], dp_result, fractional_result[1], reduced_weight_result[1]]
}
df = pd.DataFrame(data)
display(df)

,Approach,Total Calories
0,Brute Force,3256000.0
1,Dynamic Programming,3256000.0
2,Fractional Knapsack,3342800.0
3,Reduced Weight Knapsack,2554050.0


# Adjust weights to maximize calories
#### This is the solution that I find most effective because it allows me to include all the items without completely removing any of them. Instead of excluding lower-priority items, it intelligently reduces their weights while still maximizing the total calorie intake within the given weight limit. This way, I can ensure a balanced distribution of food items, making the solution more practical and efficient in real-world scenarios where completely discarding certain resources is not ideal. I really appreciate how this approach maintains variety while optimizing for the highest possible calorie count.

In [25]:
def adjust_weights_to_maximize_calories(items, max_weight):
    """Adjusts item weights to fit within max_weight while maximizing total calories without removing any item."""
    total_weight = sum(item[1] for item in items)
    if total_weight <= max_weight:
        return items, sum(item[2] for item in items)
    
    # Sort items by calorie density (calories per kg)
    items_sorted = sorted(items, key=lambda x: x[2] / x[1], reverse=True)
    
    # Reduce weights proportionally while keeping all items with at least 1 kg weight
    while total_weight > max_weight:
        for i in range(len(items_sorted)):
            if total_weight <= max_weight:
                break
            if items_sorted[i][1] > 1:  # Ensure item retains at least 1 kg
                reduction = min(10, items_sorted[i][1] - 1)  # Reduce in small steps, keeping at least 1 kg
                new_weight = items_sorted[i][1] - reduction
                new_calories = items_sorted[i][2] * (new_weight / items_sorted[i][1])
                items_sorted[i] = (items_sorted[i][0], new_weight, new_calories)
                total_weight -= reduction
    
    total_calories = sum(item[2] for item in items_sorted)
    return items_sorted, total_calories

# Define items as (name, weight, calories)
items = [
    ("Potatoes", 800, 1501600),
    ("Wheat flour", 400, 1444000),
    ("Rice", 300, 1122000),
    ("Beans (can)", 300, 690000),
    ("Tomatoes (can)", 300, 237000),
    ("Strawberry jam", 50, 130000),
    ("Peanut butter", 20, 117800)
]

max_weight = 1000

# Run the weight adjustment algorithm
adjusted_items, total_calories = adjust_weights_to_maximize_calories(items, max_weight)

print("Adjusted Item Weights and Calories:")
for item in adjusted_items:
    print(f"{item[0]}: {item[1]} kg, {int(item[2])} calories")

print("\nTotal Calories:", int(total_calories))


Adjusted Item Weights and Calories:
Peanut butter: 1 kg, 5890 calories
Rice: 70 kg, 261800 calories
Wheat flour: 180 kg, 649800 calories
Strawberry jam: 1 kg, 2600 calories
Beans (can): 80 kg, 184000 calories
Potatoes: 580 kg, 1088660 calories
Tomatoes (can): 80 kg, 63200 calories

Total Calories: 2255950
